In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
wids = pd.read_csv('repos/data/wids_reduced.csv')

In [8]:
wids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Data columns (total 81 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   encounter_id                   91713 non-null  int64  
 1   patient_id                     91713 non-null  int64  
 2   hospital_id                    91713 non-null  int64  
 3   hospital_death                 91713 non-null  int64  
 4   age                            87485 non-null  float64
 5   bmi                            88284 non-null  float64
 6   elective_surgery               91713 non-null  int64  
 7   ethnicity                      90318 non-null  object 
 8   gender                         91688 non-null  object 
 9   height                         90379 non-null  float64
 10  hospital_admit_source          70304 non-null  object 
 11  icu_admit_source               91601 non-null  object 
 12  icu_id                         91713 non-null 

In [10]:
unlabeled = pd.read_csv('repos/data/unlabeled.csv')

In [4]:
wids['apache_3j_diagnosis']

0         502.01
1         203.01
2         703.03
3        1206.03
4         601.01
          ...   
91708     501.06
91709     501.05
91710     702.01
91711     203.01
91712    1409.02
Name: apache_3j_diagnosis, Length: 91713, dtype: float64

In [5]:
# Function for transformation on apache_3j_diagnosis
def grab_main_cat(col):
    output = col.astype('str').str.split('.', n=1, expand=True)[0]
    output = output.astype(str)
    
    return output
    

In [52]:
wids['apache_3j_diagnosis'] = wids['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
wids['apache_3j_diagnosis'] = wids['apache_3j_diagnosis'].astype(str)

In [53]:
wids['apache_2_diagnosis'] = wids['apache_2_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
wids['apache_2_diagnosis'] = wids['apache_2_diagnosis'].astype(str)

In [57]:
wids.columns

Index(['encounter_id', 'patient_id', 'hospital_id', 'hospital_death', 'age',
       'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source', 'icu_admit_source', 'icu_id', 'icu_stay_type',
       'icu_type', 'pre_icu_los_days', 'readmission_status', 'weight',
       'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis',
       'apache_post_operative', 'arf_apache', 'bun_apache',
       'creatinine_apache', 'gcs_eyes_apache', 'gcs_motor_apache',
       'gcs_unable_apache', 'gcs_verbal_apache', 'glucose_apache',
       'heart_rate_apache', 'hematocrit_apache', 'intubated_apache',
       'map_apache', 'resprate_apache', 'sodium_apache', 'temp_apache',
       'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_heartrate_max',
       'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max',
       'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max',
       'd1_sysbp_min', 'd1_tem

In [58]:
wids.drop(['encounter_id', 'patient_id', 'hospital_id'], axis=1, inplace=True)

In [68]:
wids.drop(['icu_id'], axis=1, inplace=True)

In [69]:
wids = pd.get_dummies(wids)

In [59]:
X = wids.drop('hospital_death', axis=1)

In [60]:
y = wids['hospital_death']

In [64]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

wids = pd.read_csv('repos/data/wids_reduced.csv')

wids.info()

unlabeled = pd.read_csv('repos/data/unlabeled.csv')

wids['apache_3j_diagnosis']

wids['apache_3j_diagnosis'] = wids['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
wids['apache_3j_diagnosis'] = wids['apache_3j_diagnosis'].astype(str)

wids['apache_2_diagnosis'] = wids['apache_2_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
wids['apache_2_diagnosis'] = wids['apache_2_diagnosis'].astype(str)

wids.columns

wids.drop(['encounter_id', 'patient_id', 'hospital_id'], axis=1, inplace=True)

wids.drop(['icu_id'], axis=1, inplace=True)

wids = pd.get_dummies(wids)

X = wids.drop('hospital_death', axis=1)

y = wids['hospital_death']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

#convert into binary values
y_pred=clf.predict(x_test)
for i in range(0,len(y_pred)):
    if y_pred[i]>=0.1:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test, y_pred)

cm

accuracy

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

y_pred_proba=clf.predict(x_test)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_proba)

cv_results = lgb.cv(
        params,
        d_train,
        num_boost_round=100,
        nfold=7,
        metrics='auc',

        # This is what I added
        stratified=False
        )

cv_results['auc-mean']

from sklearn.model_selection import cross_validate
scores = cross_validate(lgb, x_train, y_train, cv = 7, scoring = 'roc_auc')